In [2]:
# Setup: Connect to PostgreSQL database
import os
import psycopg2
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Connect to PostgreSQL
conn = psycopg2.connect(
    host=os.getenv("DB_HOST", "localhost"),
    port=os.getenv("DB_PORT", "5432"),
    user=os.getenv("DB_USER", "postgres"),
    password=os.getenv("DB_PASSWORD"),
    database=os.getenv("DB_NAME", "unicorns_db")
)

print("✅ Successfully connected to PostgreSQL database!")

# Create cursor for executing queries
cursor = conn.cursor()

# Test connection by querying companies table
cursor.execute("SELECT * FROM companies LIMIT 5;")
companies = cursor.fetchall()

# Get column names
cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'companies';")
columns = [col[0] for col in cursor.fetchall()]

print(f"\nSample data from companies table:")
print(f"Columns: {columns}")
for row in companies:
    print(row)

✅ Successfully connected to PostgreSQL database!

Sample data from companies table:
Columns: ['index', 'company_id', 'company', 'city', 'country', 'continent']
(0, 189, 'Otto Bock HealthCare', 'Duderstadt', 'Germany', 'Europe')
(1, 848, 'Matrixport', None, 'Singapore', 'Asia')
(2, 556, 'Cloudinary', 'Santa Clara', 'United States', 'North America')
(3, 999, 'PLACE', 'Bellingham', 'United States', 'North America')
(4, 396, 'candy.com', 'New York', 'United States', 'North America')

Sample data from companies table:
Columns: ['index', 'company_id', 'company', 'city', 'country', 'continent']
(0, 189, 'Otto Bock HealthCare', 'Duderstadt', 'Germany', 'Europe')
(1, 848, 'Matrixport', None, 'Singapore', 'Asia')
(2, 556, 'Cloudinary', 'Santa Clara', 'United States', 'North America')
(3, 999, 'PLACE', 'Bellingham', 'United States', 'North America')
(4, 396, 'candy.com', 'New York', 'United States', 'North America')


![Hand with calculator](calculator.jpg)

Did you know that the average return from investing in stocks is [10% per year](https://www.nerdwallet.com/article/investing/average-stock-market-return) (not accounting for inflation)? But who wants to be average?! 

You have been asked to support an investment firm by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

You have been given access to their `unicorns` database, which contains the following tables:

## dates
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `date_joined` | The date that the company became a unicorn.  |
| `year_founded` | The year that the company was founded.       |

## funding
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| `company_id`       | A unique ID for the company.                 |
| `valuation`        | Company value in US dollars.                 |
| `funding`          | The amount of funding raised in US dollars.  |
| `select_investors` | A list of key investors in the company.      |

## industries
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `industry`     | The industry that the company operates in.   |

## companies
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| `company_id`   | A unique ID for the company.                      |
| `company`      | The name of the company.                          |
| `city`         | The city where the company is headquartered.      |
| `country`      | The country where the company is headquartered.   |
| `continent`    | The continent where the company is headquartered. |


# The output

Your query should return a table in the following format:
| industry  | year | num\_unicorns       | average\_valuation\_billions |
| --------- | ---- | ------------------- | ---------------------------- |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |

Where `industry1`, `industry2`, and `industry3` are the three top-performing industries.

In [3]:
# Explore all tables in the database
import pandas as pd

query = """
SELECT * 
FROM companies
LIMIT 10;
"""

df_companies = pd.read_sql_query(query, conn)
display(df_companies)

C:\Users\Asus\AppData\Local\Temp\ipykernel_7448\359307410.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_companies = pd.read_sql_query(query, conn)


,index,company_id,company,city,country,continent
0,0,189,Otto Bock HealthCare,Duderstadt,Germany,Europe
1,1,848,Matrixport,None,Singapore,Asia
2,2,556,Cloudinary,Santa Clara,United States,North America
3,3,999,PLACE,Bellingham,United States,North America
4,4,396,candy.com,New York,United States,North America
5,5,931,HAYDON,Shanghai,China,Asia
6,6,364,eDaili,Shanghai,China,Asia
7,7,732,CoinTracker,San Francisco,United States,North America
8,8,906,EcoFlow,Shenzhen,China,Asia
9,9,72,DJI Innovations,Shenzhen,China,Asia


In [5]:
# Main Analysis: Top 3 industries by unicorn count (2019-2021) with yearly breakdown
query = """
WITH industry_counts AS (
    SELECT i.industry, COUNT(*) AS total_unicorns
    FROM industries i
    JOIN dates d ON i.company_id = d.company_id
    WHERE EXTRACT(YEAR FROM d.date_joined::date) IN (2019, 2020, 2021)
    GROUP BY i.industry
    ORDER BY total_unicorns DESC
    LIMIT 3
),
yearly_stats AS (
    SELECT 
        i.industry,
        EXTRACT(YEAR FROM d.date_joined::date) AS year,
        COUNT(*) AS num_unicorns,
        ROUND(AVG(f.valuation) / 1e9, 2) AS average_valuation_billions
    FROM industries i
    JOIN dates d ON i.company_id = d.company_id
    JOIN funding f ON i.company_id = f.company_id
    WHERE EXTRACT(YEAR FROM d.date_joined::date) IN (2019, 2020, 2021)
    GROUP BY i.industry, EXTRACT(YEAR FROM d.date_joined::date)
)
SELECT 
    y.industry,
    y.year,
    y.num_unicorns,
    y.average_valuation_billions
FROM yearly_stats y
JOIN industry_counts t ON y.industry = t.industry
ORDER BY y.year DESC, y.num_unicorns DESC;
"""

df_result = pd.read_sql_query(query, conn)
display(df_result)

C:\Users\Asus\AppData\Local\Temp\ipykernel_7448\3564956753.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = pd.read_sql_query(query, conn)


,industry,year,num_unicorns,average_valuation_billions
0,Fintech,2021.0,138,2.75
1,Internet software & services,2021.0,119,2.15
2,E-commerce & direct-to-consumer,2021.0,47,2.47
3,Internet software & services,2020.0,20,4.35
4,E-commerce & direct-to-consumer,2020.0,16,4.00
5,Fintech,2020.0,15,4.33
6,Fintech,2019.0,20,6.80
7,Internet software & services,2019.0,13,4.23
8,E-commerce & direct-to-consumer,2019.0,12,2.58
